## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-01-10-51-03 +0000,nyt,Live Updates: Dozens Believed Dead After Fire ...,https://www.nytimes.com/live/2026/01/01/world/...
1,2026-01-01-10-43-11 +0000,nyt,Here is the latest.,https://www.nytimes.com/live/2026/01/01/world/...
2,2026-01-01-10-30-00 +0000,wsj,The $358 Billion Question for the New CEO of B...,https://www.wsj.com/finance/investing/berkshir...
3,2026-01-01-10-30-00 +0000,wsj,How Meta’s Reels Became a $50 Billion Business,https://www.wsj.com/tech/meta-reels-revenue-ad...
4,2026-01-01-10-23-01 +0000,startribune,Cities around the world welcome 2026 with thun...,https://www.startribune.com/new-zealands-auckl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2406/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,year,36
6,new,35
126,trump,34
292,guard,12
158,ukraine,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
64,2026-01-01-02-19-53 +0000,nypost,"Trump calls for ‘peace on Earth,’ vows to ‘get...",https://nypost.com/2025/12/31/us-news/trump-ca...,138
30,2026-01-01-07-42-14 +0000,nypost,"‘Several tens’ dead, about 100 injured as fire...",https://nypost.com/2026/01/01/world-news/swiss...,120
24,2026-01-01-09-00-00 +0000,cbc,"New year, new tax measures: What to expect in ...",https://www.cbc.ca/news/politics/2026-tax-meas...,117
115,2025-12-31-23-02-41 +0000,nyt,"In New Year’s Speech to Russia, Putin Says Lit...",https://www.nytimes.com/2025/12/31/world/europ...,106
16,2026-01-01-09-49-07 +0000,wapo,Fire kills several dozen at bar in Swiss ski r...,https://www.washingtonpost.com/world/2026/01/0...,104


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
64,138,2026-01-01-02-19-53 +0000,nypost,"Trump calls for ‘peace on Earth,’ vows to ‘get...",https://nypost.com/2025/12/31/us-news/trump-ca...
30,49,2026-01-01-07-42-14 +0000,nypost,"‘Several tens’ dead, about 100 injured as fire...",https://nypost.com/2026/01/01/world-news/swiss...
141,46,2025-12-31-21-20-43 +0000,nypost,"Trump pulling National Guard from Chicago, Por...",https://nypost.com/2025/12/31/us-news/trump-re...
57,38,2026-01-01-03-28-50 +0000,cbc,U.S. military says 8 killed in strikes on 5 mo...,https://www.cbc.ca/news/world/us-venezuela-dru...
103,33,2025-12-31-23-27-34 +0000,nypost,Accused Christmas killer hit with rare first-d...,https://nypost.com/2025/12/31/us-news/christma...
98,33,2025-12-31-23-37-45 +0000,nypost,Tax hike on NY workers will be needed to fund ...,https://nypost.com/2025/12/31/us-news/mamdani-...
215,33,2025-12-31-14-48-14 +0000,nypost,"Russian drone attack injures 6, including chil...",https://nypost.com/2025/12/31/world-news/russi...
90,32,2026-01-01-00-11-00 +0000,wsj,Trump’s surprise veto of a Colorado water proj...,https://www.wsj.com/politics/policy/trump-veto...
171,32,2025-12-31-19-00-05 +0000,nypost,San Fran mayor signs bill that could give blac...,https://nypost.com/2025/12/31/us-news/san-fran...
135,29,2025-12-31-21-46-20 +0000,nypost,Socialist LA official claims she won’t accept ...,https://nypost.com/2025/12/31/us-news/socialis...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
